In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 

In [3]:
billing = pd.read_csv("billing.csv") 
events = pd.read_csv("events.csv")
sessions = pd.read_csv("sessions.csv")
users = pd.read_csv('users.csv')
final_expansion_group = pd.read_csv('final_expansion_group.csv')
final_control_group = pd.read_csv('final_control_group.csv')

Revised Feature Engineering Plan: From Broad to Hyper-Specific

Your goal is still to create the final_analysis_df. The features from sessions.csv and billing.csv remain the same. But for events.csv, we will create a much more powerful and targeted set of columns.

For each user, within their specific analysis window, we will now calculate:
1. Individual Feature Adoption (The "Did they even touch it?" metrics)

This is the most crucial part. For each of the seven features, we will create a binary flag (1 if used, 0 if not). This will be the most powerful predictor in your model.

    used_jira_board: 1 or 0

    used_jira_automation: 1 or 0

    used_team_invite: 1 or 0

    used_bitbucket_integration: 1 or 0

    used_confluence_page: 1 or 0

    used_confluence_ai_assist: 1 or 0

    used_trello_export_csv: 1 or 0

2. Feature Usage Intensity (The "How much did they use it?" metrics)

For users who adopted a feature, how deeply did they engage?

    jira_board_event_count: The total count of events for this feature.

    team_invite_event_count: The count of team_invite events. This is a critical metric. A user who invites 1 person is different from one who invites 10.

    confluence_page_event_count: Total events.

    ...and so on for the other features.

3. Composite & Strategic Features (The "What combinations matter?" metrics)

This is where you can generate powerful, top-tier insights. The magic is often in the combination of features used.

    feature_adoption_count: The sum of the binary flags from section 1. This is our new, much simpler "feature diversity" score (it will be a number from 0 to 7).

    used_core_jira_suite: A binary flag that is 1 if (used_jira_board == 1 AND used_jira_automation == 1), else 0. This shows a commitment to the Jira ecosystem.

    used_core_confluence_suite: A binary flag that is 1 if (used_confluence_page == 1 AND used_confluence_ai_assist == 1), else 0.

    used_any_integration: A binary flag that is 1 if (used_bitbucket_integration == 1 OR used_trello_export_csv == 1), else 0. This signals the product is becoming embedded in their workflow.

Your Next Steps, Clarified

Your workflow remains the same, but the core task within the loop becomes much more specific.

    Loop through each user in your Expansion and Control groups.

    Filter the events table for that user and their specific analysis window.

    Calculate the ~15 new features listed above (7 binary flags, ~5 counts, 3 composite flags).

    Store these features in your final_analysis_df.

Our Hypotheses Are Now Razor-Sharp

Now, we can state our hypotheses with extreme clarity:

    Hypothesis (Team Invite): "Users in the Expansion Group have a significantly higher rate of using the team_invite feature in their pre-expansion window compared to the Control Group."

    Hypothesis (Automation Lever): "Adoption of 'power features' like jira_automation or confluence_ai_assist is a strong leading indicator of future expansion."

    Hypothesis (Ecosystem Stickiness): "Users who integrate the product with other tools (Bitbucket, Trello) are more likely to expand their seat count because the product has become a central hub for their work."

You are now in an excellent position. This level of feature specificity is what separates a generic report from a data-driven strategy document that can change how a business operates. Proceed with creating this feature set. The next phase of visualization and comparison will be incredibly insightful.

In [34]:
#5e9df73e-052e-402d-abdf-940f20f4d440
# billing[billing['user_id'] == '5e9df73e-052e-402d-abdf-940f20f4d440']
# 1. Sort by user_id and month
billing = billing.sort_values(["user_id", "month"])
# 2. High-Water Mark: cumulative max per user
billing["max_seats_to_date"] = billing.groupby("user_id")["active_seats"].cummax()
# 3. Previous High-Water Mark: shift within each user’s history
billing["prev_max_seats_to_date"] = billing.groupby("user_id")["max_seats_to_date"].shift()
# 4. True Expansion Month condition
billing["true_expansion_month"] = billing["active_seats"] > billing["prev_max_seats_to_date"]


expansion_events = billing[(billing['user_id'].isin(final_expansion_group['user_id'])) & (billing['true_expansion_month'] == True)]

first_expansion_dates = (
    expansion_events
    .groupby("user_id", as_index=False)
    .agg(first_expansion_month=("month", "min"))
)

# 3. Define pre-expansion window
# If 'month' is stored as a string like 'YYYY-MM', convert to datetime (first day of month)
first_expansion_dates["first_expansion_month"] = pd.to_datetime(first_expansion_dates["first_expansion_month"])

# The pre-expansion window is simply the prior calendar month
first_expansion_dates["pre_expansion_start"] = (first_expansion_dates["first_expansion_month"] - pd.offsets.MonthBegin(1))
first_expansion_dates["pre_expansion_end"]   = (first_expansion_dates["first_expansion_month"] - pd.offsets.MonthEnd(1))


In [35]:
first_expansion_dates

,user_id,first_expansion_month,pre_expansion_start,pre_expansion_end
0,000f0003-fbcc-4375-a9d6-1d8c63dbdf05,2024-10-01,2024-09-01,2024-09-30
1,001152fb-e29a-4cf5-878c-901af2268ad0,2025-04-01,2025-03-01,2025-03-31
2,001281b3-e61d-45e0-8551-68da908a2edf,2024-05-01,2024-04-01,2024-04-30
3,0012a970-074a-4701-b157-207958ab299c,2025-04-01,2025-03-01,2025-03-31
4,001a825f-72d1-4b7f-a2f5-f14cb5bc0714,2024-12-01,2024-11-01,2024-11-30
...,...,...,...,...
19644,ffe21a37-d8b8-4cd4-8ef1-923223caa39a,2025-06-01,2025-05-01,2025-05-31
19645,ffe7a232-fe36-4fdf-9c5d-6c9af2b8d65f,2024-03-01,2024-02-01,2024-02-29
19646,ffeab75b-1fb0-4786-9a2c-678e15bdee61,2025-05-01,2025-04-01,2025-04-30
19647,fff871bd-392e-4287-95a2-19d85aaf8196,2025-05-01,2025-04-01,2025-04-30


In [44]:
final_control_group['signup_date'] = pd.to_datetime(final_control_group['signup_date'])
final_control_group['signup_month'] = final_control_group['signup_date'].dt.to_period('M')

final_control_dates = final_control_group[['user_id', 'signup_date', 'signup_month']].copy()

# Define observation window
final_control_dates['observation_start'] = final_control_dates['signup_date']
final_control_dates['observation_end']   = final_control_dates['signup_date'] + pd.Timedelta(days=60)
